In [1]:
#превращает файлы с последовательностями картинок в файлы с последовательностями их эмбеддингов
import os
import sys
sys.path.insert(1, '../env')
sys.path.insert(1, '../agents')
sys.path.insert(1, '../')

import random
import numpy as np
from collections import deque
import keras

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import pickle
import copy

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import resnet_v2, mobilenet_v2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [28]:
_IMAGE_NET_TARGET_SIZE = (224, 224)

class Img2Vec(object):
    #benchmark:
    #resnet50 5.68 s/100 times
    #resnet_v2.ResNet101V2 Wall time: 9.56 s
    #resnet_v2.ResNet152V2 Wall time: 10.9 s
    #mobilenet_v2.MobileNetV2 Wall time: 3.9 s
    def __init__(self):
        
        #model = resnet_v2.ResNet101V2(weights='imagenet')
        model = mobilenet_v2.MobileNetV2(weights='imagenet')
        #layer_name = 'avg_pool'
        layer_name = model.layers[-2].name
        self.intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)


    def get_vec_file(self, image_path):
        """ Gets a vector embedding from an image.
        :param image_path: path to image on filesystem
        :returns: numpy ndarray
        """

        img = image.load_img(image_path, target_size=_IMAGE_NET_TARGET_SIZE)
        
        return self.get_vec_raw(x)
    def get_vec_raw(self, x):
        if len(np.shape(x))==3:
            x = np.expand_dims(x, axis=0)
        else:
            x = np.expand_dims(x, axis=1)
        x = mobilenet_v2.preprocess_input(x)
        if np.shape(x)[0]>1:
            #перевести в dataset
            x = tf.data.Dataset.from_tensor_slices(x)
            #list_dataset = list(x.as_numpy_iterator())
            #print(len(list_dataset), np.shape(list_dataset[0]))
        intermediate_output = self.intermediate_layer_model.predict(x,batch_size=100)
        #i2v.intermediate_layer_model.predict()#help is here
        return intermediate_output

In [21]:
#load data
list_imgs = pickle.load(open('../movies_save_raw/seaquest_sarsa_113440696.pkl','rb'))

In [22]:
img = list_imgs[0]
img

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [26]:
i2v = Img2Vec()

In [98]:
embedding = i2v.get_vec_raw(img)
print(embedding)
print(len(embedding))
print(np.shape(embedding))

[0.5531439  0.         0.3613987  ... 0.         0.38746122 0.09749935]
1280
(1280,)


In [7]:
%%time
#бенчмарк
for i in range(100):
    embedding = i2v.get_vec_raw(img)
#embedding = i2v.get_vec_file('./Krasivaya-koshka.jpg')
#embedding

CPU times: user 5.85 s, sys: 655 ms, total: 6.5 s
Wall time: 6.4 s


In [90]:
np.shape(embedding)

(1280,)

In [36]:
#список названий файлов
directory = '../movies_save_raw/'
files = os.listdir(directory)
i2v = Img2Vec()
i = 0
for file in files:
    print(i, len(files), pd.Timestamp.now())
    if (not ('README') in file) and not ('.ipynb_checkpoints' in file):
        list_imgs = pickle.load(open('../movies_save_raw/' + file,'rb'))
        list_imgs = np.array(list_imgs)
        list_imgs = list_imgs[:,:,:,:]
        embedding_package = i2v.get_vec_raw(list_imgs)
        #а теперь эти эмбеддинги выгрузить в другой файл
        #размерность (1001, 1280)
        with open('./movies_save_embeddings/' + file, 'wb') as f:
            pickle.dump(embedding_package, f, protocol=pickle.HIGHEST_PROTOCOL)
        i += 1

0 16 2020-08-06 23:07:16.019754
1 16 2020-08-06 23:07:34.491828
2 16 2020-08-06 23:07:45.391772
3 16 2020-08-06 23:08:02.791982
4 16 2020-08-06 23:08:20.667358
5 16 2020-08-06 23:08:38.107572
5 16 2020-08-06 23:08:38.107917
6 16 2020-08-06 23:08:56.109979
7 16 2020-08-06 23:09:15.124264
8 16 2020-08-06 23:09:33.356594
9 16 2020-08-06 23:09:51.093189
10 16 2020-08-06 23:10:09.653273
11 16 2020-08-06 23:10:27.734897
11 16 2020-08-06 23:10:27.735227
12 16 2020-08-06 23:10:45.727182
13 16 2020-08-06 23:11:03.451523


In [ ]:
embedding

In [ ]:
np.shape(embedding)